In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

In [2]:
city_data = pd.read_csv('output_data/cities.csv')

city_data.set_index("City ID", inplace=True)

city_data.head()

,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
City ID,,,,,,,,,
0,Atuona,PF,-9.8000,-139.0333,78.55,75,3,18.66,2022-02-07 13:03:52-06:00
1,Barrow,US,71.2906,-156.7887,-25.58,68,0,8.05,2022-02-07 13:03:52-06:00
2,Kavieng,PG,-2.5744,150.7967,81.64,77,100,7.63,2022-02-07 13:03:52-06:00
3,Punta Arenas,CL,-53.1500,-70.9167,60.82,93,100,6.91,2022-02-07 13:03:53-06:00
4,Khatanga,RU,71.9667,102.5000,-23.10,99,99,8.90,2022-02-07 13:03:53-06:00


In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [44]:
#Formatting for both the 'humidity heat map' and the 'hotel location map'
figure_layout = {
    'width': '900px',
    'height': '475px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

map_center = (21, 0)
zoom = 1.65

In [45]:
#Set the Latitude and Longitude as locations and Humidity as the weight
locations = city_data[["Latitude", "Longitude"]]
weight = city_data["Humidity (%)"]

#Set up empty map
fig = gmaps.figure(center = map_center, zoom_level = zoom, layout=figure_layout)

#Add heatmap layer to map
heatmap_layer = gmaps.heatmap_layer(locations, weight, dissipating = True, max_intensity = 110, point_radius = 8)
fig.add_layer(heatmap_layer)

#Display map
fig

Figure(layout=FigureLayout(border='1px solid black', height='475px', margin='0 auto 0 auto', padding='1px', wi…

In [46]:
#Narrow down data frame to ideal vacation conditions
vacay_temp_min = 75.0
vacay_temp_max = 87.5
wind_speed_max = 10.0
humidity_max = 75
cloud_max = 20.0

#Create data frame from only the ideal vacation locations
vacation_cities = city_data[(city_data["Max Temp (F)"] >= vacay_temp_min) & 
                            (city_data["Max Temp (F)"] < vacay_temp_max) &
                            (city_data["Wind Speed (mph)"] <= wind_speed_max) &
                            (city_data["Humidity (%)"] < humidity_max) &
                            (city_data["Cloudiness (%)"] < cloud_max)                           
                           ]

#Confirm that the "ideal" parameters are not too narrow
len(vacation_cities.index)

21

## Hotel Map

In [47]:
# Find closest hotel/lodging

#Create a data frame that will include hotel inforamtion
hotel_df = vacation_cities.copy()

#Add empty columns for "Hotel Name" and "Hotel Rating" to data frame
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Rating"] = ""

# Base url for gmaps API
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#Set up parameter dictionary. Note: "lodging" is listed as a "Place Type" in gmaps API documentation, "hotel" is not
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key,
}

#Loop through cities in the hotel_df data frame to look for hotels
for index, row in hotel_df.iterrows():

    #Use the city's latitude and longitude to find the nearest hotel/lodging
    city_lat = row["Latitude"]
    city_lon = row["Longitude"]

    #append lat, lon to list of parameters
    params["location"] = f"{city_lat},{city_lon}"
    
    #get data from gmaps
    response = requests.get(base_url, params=params).json()  

    try:       
        #Add first hotel result for each city to data frame        
        hotel_df.loc[index, "Hotel Name"] = response["results"][0]["name"]
        hotel_df.loc[index, "Hotel Rating"] = response["results"][0]["rating"]
        print(f"Retrieving results for city {index}: {row[0]}")
        
    except (KeyError, IndexError):
        print(f"Missing field. Skipping {row[0]}.")

#Drop data frame rows with blanks
hotel_df = hotel_df[(hotel_df["Hotel Name"] != '') & (hotel_df["Hotel Rating"] != '')]   

#Convert ratings from objects to numbers
hotel_df["Hotel Rating"] = pd.to_numeric(hotel_df["Hotel Rating"], errors='coerce')

#Only keep the top 10 rated hotels, ensures that no more than 10 hotels are pinned on the map
hotel_df = hotel_df.nlargest(10, "Hotel Rating", keep='first')

#Display the data frame
hotel_df

Missing field. Skipping Businga.
Retrieving results for city 29: General Pico
Retrieving results for city 33: Cabo San Lucas
Retrieving results for city 72: Morant Bay
Retrieving results for city 77: Veinticinco de Mayo
Retrieving results for city 110: Castro
Retrieving results for city 124: Lüderitz
Missing field. Skipping Ouallam.
Missing field. Skipping Bara.
Retrieving results for city 179: Xaxim
Retrieving results for city 278: San Clemente
Retrieving results for city 282: Graaff-Reinet
Missing field. Skipping Coahuayana Viejo.
Retrieving results for city 360: Itaqui
Missing field. Skipping Tari.
Retrieving results for city 422: Aquiraz
Retrieving results for city 426: Pochutla
Retrieving results for city 432: Lompoc
Retrieving results for city 450: Imbituba
Retrieving results for city 514: Bahía Honda
Retrieving results for city 537: San Juan


,City,Country,Latitude,Longitude,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Hotel Name,Hotel Rating
City ID,,,,,,,,,,,
422,Aquiraz,BR,-3.9014,-38.3911,86.09,71,19,9.82,2022-02-07 13:05:15-06:00,Pousada Rua Camurupim,5.0
514,Bahía Honda,CU,22.9036,-83.1592,80.02,58,5,8.43,2022-02-07 13:06:21-06:00,Casa Gualberto - Deysi,5.0
282,Graaff-Reinet,ZA,-32.2522,24.5308,78.93,60,9,7.92,2022-02-07 13:05:12-06:00,Rietjiesbos Self Catering,4.6
450,Imbituba,BR,-28.2400,-48.6703,76.89,57,11,9.80,2022-02-07 13:06:02-06:00,Praia Hotel Imbituba,4.6
33,Cabo San Lucas,MX,22.8909,-109.9124,78.62,49,0,1.01,2022-02-07 12:59:33-06:00,Hotel Tesoro Los Cabos,4.4
278,San Clemente,US,33.4270,-117.6120,79.59,28,0,5.01,2022-02-07 13:05:11-06:00,Casablanca Inn,4.3
110,Castro,BR,-24.7911,-50.0119,79.66,46,0,4.79,2022-02-07 13:04:22-06:00,CHACARA BAILLY,4.2
124,Lüderitz,NaN,-26.6481,15.1594,75.61,63,9,7.25,2022-02-07 13:04:25-06:00,LÜDERITZ NEST HOTEL,4.2
432,Lompoc,US,34.6391,-120.4579,77.13,43,0,5.75,2022-02-07 13:05:56-06:00,"Holiday Inn Express Lompoc, an IHG Hotel",4.2


In [53]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Hotel Rating</dt><dd>{Hotel Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [71]:
#Hover text information
hotel_name = hotel_df["Hotel Name"].astype(str)
hotel_name.reset_index(drop=True, inplace=True)

In [72]:
hotel_locations = hotel_df[["Latitude", "Longitude"]]

#Set up empty map
fig = gmaps.figure(layout = figure_layout, center = map_center, zoom_level = zoom)

#Add heat map layer
heatmap_layer = gmaps.heatmap_layer(locations, weight, dissipating = True, max_intensity = 110, point_radius = 8)
fig.add_layer(heatmap_layer)

#Add layer with hotel markers
marker_layer = gmaps.marker_layer(hotel_locations, info_box_content = hotel_info, hover_text = hotel_name)
fig.add_layer(marker_layer)

#Display the figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='475px', margin='0 auto 0 auto', padding='1px', wi…